# Student Dropout Analysis for School Education

In [24]:
import pandas as pd
from pymongo import MongoClient

In [25]:
# Connect to MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['student2']
collection = db['collection2']

In [26]:
# Load the dataset
import pandas as pd
data = pd.read_csv("C:\\Users\\Dhairya Hindoriya\\BDT Mini project\\SIH3-updated1.csv")
data

,Course,Age at enrollment,Daytime/evening attendance,Area,Caste,Displaced,Educational special needs,Debtor,Tuition fees up to date,Gender,Scholarship holder,Curricular units 1st sem (grade),Curricular units 2nd sem (grade),Unemployment rate,Inflation rate,GDP,Target,Previous Qualification
0,Physics,16,1,Sikkim,Slaute,1,0,0,1,Male,0,0.000000,0.000000,10.8,1.4,1.74,Dropout,8
1,Sociology,10,1,Rajasthan,Ambacam,1,0,0,0,Male,0,14.000000,13.666667,13.9,-0.3,0.79,Graduate,6
2,English,12,1,Uttaranchal,Chawhan,1,0,0,0,Male,0,0.000000,0.000000,10.8,1.4,1.74,Dropout,8
3,Music,14,1,Uttar Pradesh,Rajput,1,0,0,1,Female,0,13.428571,12.400000,9.4,-0.8,-3.12,Graduate,6
4,Chemistry,11,0,Rajasthan,Raghuransi,0,0,0,1,Female,0,12.333333,13.000000,13.9,-0.3,0.79,Graduate,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4421,Psychology,13,1,Uttaranchal,Asooy,1,0,0,1,Female,1,14.912500,13.500000,13.9,-0.3,0.79,Dropout,6
4422,Economics,15,1,Haryana,Raghuransi,1,0,0,1,Female,1,13.800000,12.000000,9.4,-0.8,-3.12,Graduate,6
4423,Music,11,1,Delhi,Marer,1,0,0,1,Female,0,11.666667,13.000000,12.7,3.7,-1.70,Graduate,7
4424,Maths,14,0,Goa,Kanojia,0,1,1,0,Female,1,11.000000,10.000000,10.8,1.4,0.79,Dropout,9


In [27]:
print(data.dtypes)

Course                               object
Age at enrollment                     int64
Daytime/evening attendance            int64
Area                                 object
Caste                                object
Displaced                             int64
Educational special needs             int64
Debtor                                int64
Tuition fees up to date               int64
Gender                               object
Scholarship holder                    int64
Curricular units 1st sem (grade)    float64
Curricular units 2nd sem (grade)    float64
Unemployment rate                   float64
Inflation rate                      float64
GDP                                 float64
Target                               object
Previous Qualification                int64
dtype: object


In [28]:
# Convert DataFrame to list of dictionaries
data_dict = data.to_dict(orient='records')

# Insert data into MongoDB
collection.insert_many(data_dict)

InsertManyResult([ObjectId('6565b1b819a1c6dfdc2d5eda'), ObjectId('6565b1b819a1c6dfdc2d5edb'), ObjectId('6565b1b819a1c6dfdc2d5edc'), ObjectId('6565b1b819a1c6dfdc2d5edd'), ObjectId('6565b1b819a1c6dfdc2d5ede'), ObjectId('6565b1b819a1c6dfdc2d5edf'), ObjectId('6565b1b819a1c6dfdc2d5ee0'), ObjectId('6565b1b819a1c6dfdc2d5ee1'), ObjectId('6565b1b819a1c6dfdc2d5ee2'), ObjectId('6565b1b819a1c6dfdc2d5ee3'), ObjectId('6565b1b819a1c6dfdc2d5ee4'), ObjectId('6565b1b819a1c6dfdc2d5ee5'), ObjectId('6565b1b819a1c6dfdc2d5ee6'), ObjectId('6565b1b819a1c6dfdc2d5ee7'), ObjectId('6565b1b819a1c6dfdc2d5ee8'), ObjectId('6565b1b819a1c6dfdc2d5ee9'), ObjectId('6565b1b819a1c6dfdc2d5eea'), ObjectId('6565b1b819a1c6dfdc2d5eeb'), ObjectId('6565b1b819a1c6dfdc2d5eec'), ObjectId('6565b1b819a1c6dfdc2d5eed'), ObjectId('6565b1b819a1c6dfdc2d5eee'), ObjectId('6565b1b819a1c6dfdc2d5eef'), ObjectId('6565b1b819a1c6dfdc2d5ef0'), ObjectId('6565b1b819a1c6dfdc2d5ef1'), ObjectId('6565b1b819a1c6dfdc2d5ef2'), ObjectId('6565b1b819a1c6dfdc2d5e

# Data Preprocessing

In [29]:
# Rename the 'Age at enrollment' field to 'Age'
collection.update_many({}, {"$rename": {"Age at enrollment": "Age"}})

UpdateResult({'n': 4426, 'nModified': 4426, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [30]:
# Drop unnecessary columns
collection.update_many({}, {"$unset": {"Educational special needs": "", "Unemployment rate": "", "Inflation rate": ""}})

UpdateResult({'n': 4426, 'nModified': 4426, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [31]:
# Find documents where 'field_name' does not exist
null_documents = collection.find({"field_name": {"$exists": False}})


In [32]:
# Find distinct values of 'Target' field
distinct_targets = collection.distinct("Target")
print(distinct_targets)

['Dropout', 'Enrolled', 'Graduate']


In [33]:
# Aggregate to count occurrences of each 'Target' value
target_counts_pipeline = [
    {"$group": {"_id": "$Target", "count": {"$sum": 1}}}
]
target_counts = list(collection.aggregate(target_counts_pipeline))
print(target_counts)

[{'_id': 'Dropout', 'count': 1422}, {'_id': 'Graduate', 'count': 2210}, {'_id': 'Enrolled', 'count': 794}]


In [34]:
# Example: Find a few documents to check the 'Target' values
sample_documents = collection.find().limit(5)
for doc in sample_documents:
    print(doc)


{'_id': ObjectId('6565b1b819a1c6dfdc2d5eda'), 'Course': 'Physics', 'Daytime/evening attendance': 1, 'Area': 'Sikkim', 'Caste': 'Slaute', 'Displaced': 1, 'Debtor': 0, 'Tuition fees up to date': 1, 'Gender': 'Male', 'Scholarship holder': 0, 'Curricular units 1st sem (grade)': 0.0, 'Curricular units 2nd sem (grade)': 0.0, 'GDP': 1.74, 'Target': 'Dropout', 'Previous Qualification': 8, 'Age': 16}
{'_id': ObjectId('6565b1b819a1c6dfdc2d5edb'), 'Course': 'Sociology', 'Daytime/evening attendance': 1, 'Area': 'Rajasthan', 'Caste': 'Ambacam', 'Displaced': 1, 'Debtor': 0, 'Tuition fees up to date': 0, 'Gender': 'Male', 'Scholarship holder': 0, 'Curricular units 1st sem (grade)': 14.0, 'Curricular units 2nd sem (grade)': 13.66666667, 'GDP': 0.79, 'Target': 'Graduate', 'Previous Qualification': 6, 'Age': 10}
{'_id': ObjectId('6565b1b819a1c6dfdc2d5edc'), 'Course': 'English', 'Daytime/evening attendance': 1, 'Area': 'Uttaranchal', 'Caste': 'Chawhan', 'Displaced': 1, 'Debtor': 0, 'Tuition fees up to da